In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [5]:
n_actions = 10
n_features = 20
n_data_list = [250, 500, 1000, 2000, 4000, 8000]

In [25]:
n_data = n_data_list[1]

x = np.random.normal(size=(n_data, n_features))
Wx = np.random.normal(size=(n_actions, n_features))
Bx = np.random.normal(size=(n_actions, 1))

Wxa = np.random.normal(size=(n_features, n_actions, 1))

def sigmoid(x):
    return 1./(1.+np.exp(x))

def feature2action(x):
    return sigmoid(Wx@x.T+Bx).T

def action2reword(x, a):
    return Wxa

In [21]:
x.shape

(500, 20)

In [22]:
feature2action(x).shape

(500, 10)

In [26]:
Wxa.shape

(20, 10)